In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install transformers
!pip install datasets
!pip install torch
!pip install langchain tiktoken
!pip install dask tensorflow
!pip install wandb

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114

### Pre Train

### Start directly from here

In [ ]:
import datasets
from datasets import Dataset
from transformers import BertTokenizer, DataCollatorForLanguageModeling
import pickle
from transformers import AutoModelForMaskedLM, AutoTokenizer, TrainingArguments
import pandas as pd
import os
# from torch.utils.data import Dataset
import torch
import tiktoken
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

import gc

from huggingface_hub import notebook_login

notebook_login()

import wandb
from wandb.keras import WandbCallback
import math

%wandb login

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
#  wandb.init(
#       # set the wandb project where this run will be logged
#       project="clinical-context",
      
#       # track hyperparameters and run metadata
#       config={
#       "chunk_size": 128,
#       "architecture": "BIOBERT",
#       "dataset": "MIMIC-4",
#       "epochs": 10,
#       "shard" : 1
#       }
#   )

In [ ]:


import pandas as pd

def concat_columns(df):
    # Concatenate all columns into a single column 'combined'
    df['combined_text'] = df.apply(lambda x: ' '.join(x.astype(str)), axis=1)
    
    # Drop the original individual columns
    df = df.drop(columns=df.columns[:-1])

    # Rename the 'combined' column to 'text'
    df = df.rename(columns={'combined': 'text'})
    
    return df

def save_chunks(dataframe, chunk_size, output_dir):
    print("Splitting dataframe split size %s" %chunk_size)
    os.makedirs(output_dir, exist_ok=True)
    num_rows = len(dataframe)
    num_chunks = (num_rows + chunk_size - 1) // chunk_size

    for chunk_idx in range(num_chunks):
        start_idx = chunk_idx * chunk_size
        end_idx = min(start_idx + chunk_size, num_rows)

        chunk_data = dataframe.iloc[start_idx:end_idx]
        chunk_data.to_csv(os.path.join(output_dir, f"chunk_{chunk_idx}.csv"), index=False)
        print("Split Saved : %s " %chunk_idx)

      

def prepare_dataset(csv_file, output_dir, text_columns = ['text'], chunk_size=300, split_size=1000000):
    
    df = pd.read_csv(csv_file)
    print("Processing file %s shape %s " %(csv_file,df.shape))
    columns_to_drop = [column for column in df.columns if column not in text_columns]
    df.drop(columns_to_drop, axis=1)
    print("Dropped non text columns. Remaining text columns %s " %df.columns)
    df = concat_columns(df)

    chunk_size = 300

    # text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=chunk_size, chunk_overlap=20)
    text_splitter = RecursiveCharacterTextSplitter(
        # Set a really small chunk size, just to show.
        chunk_size = chunk_size,
        chunk_overlap  = 20,
        length_function = len,
    )

    print("Chunking start")

    # Apply the split function to the 'text' column and store the resulting words in a list
    chunks_list = df['text'].apply(lambda x: text_splitter.split_text(x))


    # Flatten the list of lists
    chunks = [chunk for sublist in chunks_list for chunk in sublist]

    # Create a new DataFrame with the words
    chunks_df = pd.DataFrame(chunks, columns=['text'])

    print("Chunked data frame shape %s " %chunks_df.shape)

    split_size = 1000000  # Adjust this value according to your system's memory capacity
    save_chunks(chunks_df, split_size, output_dir)
    chunks_df.to_csv()
    return chunks_df


# Check token distribution
def get_token_length(text):
  # Global variables
  tokenizer = tiktoken.get_encoding(
      "cl100k_base"
  )  # The encoding scheme to use for tokenization
  return len(tokenizer.encode(text, disallowed_special=()))


def tokenize_csv(chunk_file, tokenized_dir,split_size=500000):
    os.makedirs(tokenized_dir, exist_ok=True)
    print("Tokenizing start : file %s " %chunk_file)
    idx = 0
    tokenized_files = os.listdir(tokenized_dir)
    for chunk in pd.read_csv(chunk_file, chunksize=split_size):
      tokenized_chunk_name = "tokenized_{0}_{1}".format(os.path.basename(chunk_file),idx)
      if tokenized_chunk_name in tokenized_files:
          print("Token data set already present for chunk. Skipping.")
          idx = idx + 1
          continue
      shard = Dataset.from_pandas(chunk)
      tokenized_shard = shard.map(tokenize_function, batched=True, batch_size=32, num_proc=4)
      tokenized_shard.save_to_disk(os.path.join(tokenized_dir, "tokenized_{0}_{1}".format(os.path.basename(chunk_file),idx)))
      print("Tokenized chunk : idx %s : shape %s" %(idx,chunk.shape))
      idx = idx+1
      try:
        del chunk
        del shard
        del tokenized_shard
        gc.collect()
      except Exception as e:
        pass 
    
    print("Tokenizing complete : file %s" %chunk_file)
    

In [ ]:

def tokenize_function(examples):
    try:
      # print(examples)
      examples_text = [text or " " for text in examples["text"]]
      return tokenizer(examples_text, padding="max_length", truncation=True,max_length=chunk_size)
    except Exception as e:
      print(e)
      # # print(examples)
      # return tokenizer(examples, padding="max_length", truncation=True,max_length=512)

      
      # print("Exception in tokenizer : %s " %e)

def tokenize_files(files_dir, limit=1):
    tokenized_dir = os.path.join(files_dir, "tokenized_dataset_{0}".format(chunk_size))
    os.makedirs(tokenized_dir, exist_ok=True)
    print("Tokenzed data director %s " %tokenized_dir)
    print("Tokenizing files in %s " %files_dir)
    chunk_files = sorted(os.listdir(files_dir))
    tokenized_files = os.listdir(tokenized_dir)
    print("Tokenized files %s " %tokenized_files)
    stop_limit = 0
    for chunk in chunk_files:
      try:
        if stop_limit > limit:
          break
        chunk_file = os.path.join(files_dir, chunk)
        tokenized_file_name = "tokenized_{0}".format(chunk)
        tokenized_file = os.path.join(tokenized_dir, tokenized_file_name)
        if tokenized_file_name in tokenized_files:
          print("Token data set already present. Skipping.")
          continue

        # chunk_data = pd.read_csv(chunk_file)
        tokenize_csv(chunk_file, tokenized_dir)
        stop_limit = stop_limit + 1
        
        # gc.collect()


      except Exception as error:
        print("Tokenising error in file %s" %error)


    return tokenized_dir






In [ ]:
#Tokenizing code

wandb.init(
      # set the wandb project where this run will be logged
      project="clinical-context",
      
      # track hyperparameters and run metadata
      config={
      "task" : "tokenizer",
      "dataset" : "discharge-notes"
      }
  )

chunk_size = 128
model_name = "dmis-lab/biobert-base-cased-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
dataset_dir = "/content/drive/MyDrive/ClinicalContext/discharge_chunks"
tokenized_dir = tokenize_files(dataset_dir, limit=100)


wandb: Currently logged in as: js12962 (clinical-context). Use `wandb login --relogin` to force relogin


Tokenzed data director /content/drive/MyDrive/ClinicalContext/discharge_chunks/tokenized_dataset_128 
Tokenizing files in /content/drive/MyDrive/ClinicalContext/discharge_chunks 
Tokenized files ['tokenized_chunk_0.csv_0', 'tokenized_chunk_0.csv_1', 'tokenized_chunk_1.csv_0', 'tokenized_chunk_1.csv_1', 'tokenized_chunk_10.csv_0', 'tokenized_chunk_10.csv_1', 'tokenized_chunk_11.csv_0', 'tokenized_chunk_11.csv_1', 'tokenized_chunk_12.csv_0', 'tokenized_chunk_12.csv_1', 'tokenized_chunk_13.csv_0', 'tokenized_chunk_13.csv_1', 'tokenized_chunk_14.csv_0', 'tokenized_chunk_14.csv_1', 'tokenized_chunk_15.csv_0', 'tokenized_chunk_2.csv_0', 'tokenized_chunk_2.csv_1', 'tokenized_chunk_3.csv_0', 'tokenized_chunk_3.csv_1', 'tokenized_chunk_4.csv_0', 'tokenized_chunk_4.csv_1', 'tokenized_chunk_5.csv_0', 'tokenized_chunk_5.csv_1', 'tokenized_chunk_6.csv_0', 'tokenized_chunk_6.csv_1', 'tokenized_chunk_7.csv_0', 'tokenized_chunk_7.csv_1', 'tokenized_chunk_8.csv_0', 'tokenized_chunk_8.csv_1', 'tokenized

In [ ]:
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
import tensorflow as tf
from transformers import AutoModelForMaskedLM, AutoTokenizer, TrainingArguments, Trainer
from transformers import TFAutoModelForMaskedLM
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import DataCollatorForLanguageModeling
import math


chunk_size = 128


model_name = "Jaikar/biobert-base-cased-v1.1-finetuned-clinical-context"

tokenizer = AutoTokenizer.from_pretrained("Jaikar/biobert-base-cased-v1.1-finetuned-clinical-context")

model = TFAutoModelForMaskedLM.from_pretrained("Jaikar/biobert-base-cased-v1.1-finetuned-clinical-context")

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)


dataset_dir = "/content/drive/MyDrive/ClinicalContext/discharge_chunks"

tokenized_dir = os.path.join(dataset_dir, "tokenized_dataset_{0}".format(chunk_size))


tokenized_files = os.listdir(tokenized_dir)[6:]

# first_file = [tokenized_files[0]]
# first_file.extend(tokenized_files[6:])
# tokenized_files = first_file
wandb.init(
      # set the wandb project where this run will be logged
      project="clinical-context",
      
      # track hyperparameters and run metadata
      config={
      "chunk_size": 128,
      "architecture": "BIOBERT",
      "dataset": "MIMIC-4",
      "epochs": 1,
      "weight_decay_rate" : 0.01
      }
  )

for idx in range(len(tokenized_files[6:])):
  print("Training on shard %s " %tokenized_files[idx] )
  # wandb.log("Training on shard %s " %tokenized_files[idx] )
  
  tokenized_shard_path = os.path.join(tokenized_dir,tokenized_files[idx])

  tokenized_shard = datasets.load_from_disk(tokenized_shard_path)

  len(tokenized_shard)

  tokenized_shard.remove_columns(['text'])
  tokenized_shard = tokenized_shard.add_column('labels', tokenized_shard['input_ids'] )
  downsampled_dataset = tokenized_shard.train_test_split(test_size=0.1, seed=42)

  if idx == 0:
    print("Creating train and eval")
    tf_train_dataset = model.prepare_tf_dataset(
        downsampled_dataset["train"],
        collate_fn=data_collator,
        shuffle=True,
        batch_size=32,
    )

    tf_eval_dataset = model.prepare_tf_dataset(
        downsampled_dataset["test"],
        collate_fn=data_collator,
        shuffle=False,
        batch_size=32,
    )
    continue

  else:
    tf_train_dataset = model.prepare_tf_dataset(
        tokenized_shard,
        collate_fn=data_collator,
        shuffle=True,
        batch_size=32,
    )


  num_train_steps = len(tf_train_dataset)
  optimizer, schedule = create_optimizer(
      init_lr=2e-5,
      num_warmup_steps=1_000,
      num_train_steps=num_train_steps,
      weight_decay_rate=0.01,
  )
  model.compile(optimizer=optimizer)

  # Train in mixed-precision float16
  tf.keras.mixed_precision.set_global_policy("mixed_float16")

  # model_name = model_name.split("/")[-1]
  callback = PushToHubCallback(
      output_dir=f"{model_name}", tokenizer=tokenizer
  )

  
  # eval_loss = model.evaluate(tf_eval_dataset)
  # print(f"Perplexity before training : {math.exp(eval_loss):.2f}")
  model.fit(tf_train_dataset, validation_data=tf_eval_dataset, callbacks=[callback, WandbCallback()], epochs=1)
  eval_loss = model.evaluate(tf_eval_dataset)
  print(f"perplexity %s {math.exp(eval_loss):.2f} shard idx {idx}")
  wandb.log({"perplexity" : math.exp(eval_loss), "shard idx" : idx})


All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at Jaikar/biobert-base-cased-v1.1-finetuned-clinical-context.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:388: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow model

In [ ]:
eval_loss = model.evaluate(tf_eval_dataset)
print(f"Perplexity: {math.exp(eval_loss):.2f}") 

### PyTorch